In [1]:
import os
import sys
import copy
pdir = os.path.dirname(os.getcwd())
sys.path.append(pdir)

import torch
import numpy as np

import utils
import wandb

model_dir = 'models'
data_dir = 'tree_points.pt'

In [2]:
trees_data = torch.load(data_dir)

In [3]:
sweep_config = {
  "name":"laser-trees-bayes",
  "method":"bayes",
  "metric":{
      "name":"Best_acc",
      "goal":"maximize"
  },
  "parameters":{
    "loss_fn":{
      "values":["smooth-loss", "cross-entropy"]
    },
    "lr_init":{
      "min": 0.0001,
      "max": 0.001
    },
    "lr_step":{
      "min": 10,
      "max": 100
    },
    "lr_gamma":{
      "min": 0.1,
      "max": 0.9
    },
    "max_rotation":{
      "min":0.0,
      "max":2*np.pi
    },
    "max_translation":{
      "min":0.0,
      "max":2.0
    },
    "min_scale":{
      "min":0.3,
      "max":1.0
    },
    "max_scale":{
      "min":1.0,
      "max":2.0
    }
  }
}

In [4]:
def interface_to_train():
    with wandb.init() as run:
        params={
        "dataset_type":type(trees_data),
        "batch_size":128,
        "validation_split":.15,
        "test_split":.15,
        "shuffle_dataset":True,
        "random_seed":0,
        "learning_rate":[wandb.config["lr_init"], wandb.config["lr_step"], wandb.config["lr_gamma"]],  #[init, step_size, gamma] for scheduler
        "momentum":0.9, #Only used for sgd
        "epochs":150,
        "loss_fn":wandb.config["loss_fn"],
        "optimizer":"adam",
        "voting":"None",
        "train_sampler":"balanced",

        "model":"SimpleView",

        "image_dim":256,
        "camera_fov_deg":90,
        "f":1,
        "camera_dist":1.4,
        "depth_averaging":"min",
        "soft_min_k":50,
        "num_views":6,

        "transforms":['rotation','translation','scaling'], #,'translation'
        "min_rotation":0,
        "max_rotation":wandb.config["max_rotation"],
        "min_translation":0,
        "max_translation":wandb.config["max_translation"],
        "jitter_std":0,
        "min_scale":wandb.config["min_scale"],
        "max_scale":wandb.config["max_scale"],

        "species":["QUEFAG", "PINNIG", "QUEILE", "PINSYL", "PINPIN"],
        "data_resolution":"2.5cm"
        }
        
        utils.train(data_dir=data_dir, model_dir=model_dir, params=params, init_wandb=False)
    return

In [5]:
sweep_id = 'mjsa609h'
#sweep_id = wandb.sweep(sweep_config, project='laser-trees-bayes')

count = 3
wandb.agent(sweep_id, project='laser-trees-bayes', function=interface_to_train, count=count)

Create sweep with ID: mjsa609h
Sweep URL: https://wandb.ai/mja2106/laser-trees-bayes/sweeps/mjsa609h


wandb: Agent Starting Run: 2305j66z with config:
wandb: 	loss_fn: smooth-loss
wandb: 	lr_gamma: 0.4831816409822419
wandb: 	lr_init: 0.0003470958352815794
wandb: 	lr_step: 97
wandb: 	max_rotation: 2.9060673567660684
wandb: 	max_scale: 1
wandb: 	max_translation: 1
wandb: 	min_scale: 0.3456991321340901
wandb: Currently logged in as: mja2106 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Config item 'loss_fn' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_rotation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_translation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_scale' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_scale' was locked by 'sweep' (ignored update).


QUEFAG     1116
PINNIG      581
QUEILE      364
PINSYL      277
PINPIN      140
NA            2
JUNIPE        2
QUERCUS       2
DEAD          1
Name: sp, dtype: int64
Species:  ['DEAD', 'JUNIPE', 'NA', 'PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE', 'QUERCUS']
Labels:  tensor([8, 3, 6,  ..., 7, 3, 6])
Total count:  2485
Removing: DEAD
Removing: JUNIPE
Removing: NA
Removing: QUERCUS
Train Dataset:
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Validation Dataset (should match):
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Shuffling dataset...
Using balanced sampling...


wandb: Ctrl + C detected. Stopping sweep.
